# Arcanite Cookbook

This notebook walks through the complete Arcanite pipeline:

1. **Setup** - Load deck and spreads
2. **Create a Reading** - Draw cards for a spread
3. **Layer 1: Deterministic Assembly** - Assemble interpretations using RAG mapping
4. **Generate PDF** - Render a deterministic-only PDF
5. **Optional: Question Classification** - Auto-detect question type
6. **Layer 2: LLM Synthesis** - Generate cohesive narrative
7. **Generate PDF with Synthesis** - Full reading report

## 1. Setup

First, let's load the tarot deck and available spreads.

In [1]:
from arcanite.core import TarotDeck, QuestionType
from arcanite.core.spread import SpreadRegistry

# Load the 78-card tarot deck
deck = TarotDeck.load()
print(f"Loaded: {deck}")

# Load spread definitions
spreads = SpreadRegistry.from_config()
print(f"Available spreads: {spreads.list_spreads()}")

Loaded: TarotDeck(78 cards)
Available spreads: ['celtic-cross', 'five-card-cross', 'four-card-decision', 'horseshoe-apex', 'horseshoe-traditional', 'mind-body-spirit', 'past-present-future', 'relationship-spread', 'single-focus', 'situation-action-outcome', 'year-ahead']


Let's look at a specific spread to understand its structure:

In [2]:
# Examine the Past-Present-Future spread
spread = spreads.load_spread("past-present-future")

print(f"Spread: {spread.name}")
print(f"Description: {spread.description}")
print(f"\nPositions:")
for i, pos in enumerate(spread.positions):
    print(f"  {i+1}. {pos.name}")
    print(f"     RAG mapping: {pos.rag_mapping}")
    print(f"     {pos.short_description}")

Spread: 3 Cards (Past, Present, Future)
Description: Classic timeline spread showing the flow of energy

Positions:
  1. Past
     RAG mapping: temporal_positions.past
     Past influences affecting the current situation
  2. Present
     RAG mapping: temporal_positions.present
     Current energies and circumstances surrounding you
  3. Future
     RAG mapping: temporal_positions.future
     The likely outcome or direction you're heading


## 2. Create a Reading

Now let's create a reading by drawing cards for our spread.

In [3]:
from arcanite.reading import create_reading

# Create a reading with a specific question
reading = create_reading(
    deck=deck,
    spread_id="past-present-future",
    question="What do I need to know about my creative projects?",
    question_type=QuestionType.CAREER,  # We know it's career-related
    allow_reversals=True,
    seed=42,  # Optional: for reproducible results
)

print(f"Reading ID: {reading.id}")
print(f"Question: {reading.question}")
print(f"\nCards drawn:")
for card in reading.drawn_cards:
    print(f"  {card.position_name}: {card.card_name} ({card.orientation.value})")

Reading ID: 20260226_185328
Question: What do I need to know about my creative projects?

Cards drawn:
  Past: Knight of Pentacles (upright)
  Present: Queen of Pentacles (reversed)
  Future: Three of Swords (upright)


## 3. Layer 1: Deterministic Assembly

This is the heart of Arcanite's RAG system. The `assemble_context` function:
- Takes each drawn card
- Uses the position's `rag_mapping` to navigate into the card's JSON
- Extracts the position-specific interpretation
- Optionally includes question-context variations
- Finds relationships between drawn cards

**No LLM involved** - this is pure deterministic assembly of pre-curated content.

In [4]:
from arcanite.reading import assemble_context

# Assemble the context (Layer 1)
context = assemble_context(reading, deck)

print(f"Assembled {len(context.card_interpretations)} card interpretations")
print(f"Found {len(context.relationships)} card relationships")

# Look at one card's assembled interpretation
card = context.card_interpretations[0]
print(f"\n--- {card.card_name} in {card.position_name} ---")
print(f"Position description: {card.position_description[:100]}...")
print(f"\nInterpretation: {card.position_interpretation}")
print(f"\nKeywords: {', '.join(card.position_keywords)}")

Assembled 3 card interpretations
Found 0 card relationships

--- Knight of Pentacles in Past ---
Position description: What experiences, decisions, patterns, or energies from your past are directly influencing your curr...

Interpretation: Past dedication and consistent efforts have laid a strong foundation for your current stability and growth. Your reliable nature shaped your progress.

Keywords: foundational work, consistent past, habits established, missed flexibility


The assembled context can be rendered as markdown (useful for debugging or deterministic-only use):

In [5]:
# View the full assembled context as markdown
print(context.to_markdown()[:2000])  # First 2000 chars

# Tarot Reading: 3 Cards (Past, Present, Future)

**Question:** What do I need to know about my creative projects?

**Question Type:** career

## Cards Drawn

### Position 1: Past
**Knight of Pentacles** (upright)

*Position meaning:* What experiences, decisions, patterns, or energies from your past are directly influencing your current situation? This could be recent events, learned behaviors, past relationships, or even ancestral patterns. What foundation - positive or challenging - was laid that brings you to this moment?

**Interpretation:** Past dedication and consistent efforts have laid a strong foundation for your current stability and growth. Your reliable nature shaped your progress.

*Keywords:* foundational work, consistent past, habits established, missed flexibility

**Career Context:** Success through consistent work and methodical planning. Opportunities grow from steady professional dedication.

---

### Position 2: Present
**Queen of Pentacles** (reversed)

*Position 

## 4. Generate PDF (Deterministic Only)

Even without LLM synthesis, we can generate a beautiful PDF report.

In [ ]:
from pathlib import Path
from arcanite.render import render_reading_to_pdf

# Get layout positions from the spread (for visual positioning)
layout_positions = [
    (pos.x, pos.y, pos.rotation)
    for pos in spread.layout.positions
]

# Render to PDF
output_path = Path("reading_deterministic.pdf")
render_reading_to_pdf(
    reading=context,  # AssembledContext = deterministic only
    output_path=output_path,
    title="Your Creative Journey",
    layout_positions=layout_positions,
)

print(f"PDF generated: {output_path}")
print(f"Size: {output_path.stat().st_size:,} bytes")
print("\nThis PDF includes:")
print("  - Spread visualization")
print("  - Card-by-card interpretations")
print("  - NO synthesis section (deterministic only)")

PDF generated: reading_deterministic.pdf
Size: 36,879 bytes

This PDF includes:
  - Spread visualization
  - Card-by-card interpretations
  - NO synthesis section (deterministic only)


## 5. Optional: Question Classification

If you don't know the question type upfront, Arcanite can classify it using an LLM.

**Note:** This requires an LLM provider to be configured (API key, etc.)

In [ ]:
import os
from arcanite.interpretation import QuestionClassifier
from arcanite.interpretation.llm import AnthropicProvider

# Check if API key is available
if not os.environ.get("ANTHROPIC_API_KEY"):
    print("Skipping: Set ANTHROPIC_API_KEY to use question classification")
else:
    # Create an LLM provider
    provider = AnthropicProvider(
        model="claude-sonnet-4-20250514",
        api_key=os.environ["ANTHROPIC_API_KEY"],
    )

    # Classify a question
    classifier = QuestionClassifier(provider=provider)

    test_questions = [
        "Will I find love this year?",
        "Should I take this job offer?",
        "What's blocking my spiritual growth?",
        "How can I improve my finances?",
    ]

    for q in test_questions:
        result = await classifier.classify(q)
        print(f"{result.value:10} <- {q}")

love       <- Will I find love this year?
career     <- Should I take this job offer?
spiritual  <- What's blocking my spiritual growth?
financial  <- How can I improve my finances?


## 6. Layer 2: LLM Synthesis

Now let's use an LLM to weave the assembled context into a cohesive narrative.

The synthesizer:
- Takes the assembled context (Layer 1 output)
- Loads a "tradition" prompt (defines the reading style)
- Sends everything to the LLM
- Returns a flowing, narrative interpretation

**Note:** Requires an LLM provider.

In [ ]:
import os
from arcanite.interpretation import ReadingSynthesizer
from arcanite.interpretation.llm import AnthropicProvider

if not os.environ.get("ANTHROPIC_API_KEY"):
    print("Skipping: Set ANTHROPIC_API_KEY to use LLM synthesis")
    synthesized = None
else:
    # Create provider
    provider = AnthropicProvider(
        model="claude-sonnet-4-20250514",
        api_key=os.environ["ANTHROPIC_API_KEY"],
    )

    # Create synthesizer with a tradition
    synthesizer = ReadingSynthesizer(
        provider=provider,
        tradition="intuitive",  # Warm, accessible style
    )

    # Synthesize!
    synthesized = await synthesizer.synthesize(reading, context)

    print(f"Tradition: {synthesized.tradition}")
    print(f"Model: {synthesized.model_used}")
    print(f"Tokens: {synthesized.tokens_used}")
    print(f"\n{'='*60}\n")
    print(synthesized.synthesis)

Tradition: intuitive
Model: claude-sonnet-4-20250514
Tokens: 1292


## Your Creative Journey: A Story of Growth Through Challenge

Your creative projects are calling you toward a profound transformation - one that honors your past dedication while asking you to release what no longer serves your authentic expression.

**The Journey**

Looking at your past, the Knight of Pentacles reveals the beautiful foundation you've built through consistent, methodical work on your creative endeavors. You've been the reliable one, showing up day after day, putting in the hours, building your skills brick by brick. This steady dedication has given you real competency and a solid base to work from. However, this same methodical approach may have also created some rigidity in how you approach your creativity - perhaps you've become so focused on the "right" way to do things that you've lost some of the spontaneous magic that initially drew you to your creative work.

Right now, the reversed Queen of Pe

### Try a Different Tradition

The "kate-signature" tradition is more analytical and psychologically-focused:

In [ ]:
if os.environ.get("ANTHROPIC_API_KEY"):
    # Try the kate-signature tradition
    kate_synthesizer = ReadingSynthesizer(
        provider=provider,
        tradition="kate-signature",  # "Compassionate scalpel" style
    )

    kate_reading = await kate_synthesizer.synthesize(reading, context)

    print(f"Tradition: {kate_reading.tradition}")
    print(f"\n{'='*60}\n")
    print(kate_reading.synthesis)

Tradition: kate-signature


## Opening / Card Interpretations

Your creative journey reads like a textbook case of what happens when your survival strategies start suffocating your soul. The Knight of Pentacles in your past shows you've been *that person*—the one who shows up, does the work, builds the foundation brick by methodical brick. You've probably been praised for your reliability, your consistency, your ability to deliver. And honestly? That foundation is solid as hell. But here's the thing about Knights: they're so focused on the destination that they can miss the entire landscape around them.

Fast-forward to your present, and we've got the Queen of Pentacles reversed—which is basically your nurturing, creative self having an existential meltdown. You're overwhelmed, disconnected from what actually feeds your soul, and probably running on fumes while telling everyone (including yourself) that you're "fine." This isn't about being ungrateful for your success; this is about re

## 7. Generate PDF with Synthesis

Now let's generate a complete PDF with the LLM-synthesized reading.

In [ ]:
if synthesized:
    output_path = Path("reading_synthesized.pdf")
    render_reading_to_pdf(
        reading=synthesized,  # SynthesizedReading includes the narrative
        output_path=output_path,
        title="Your Creative Journey",
        layout_positions=layout_positions,
    )

    print(f"PDF generated: {output_path}")
    print(f"Size: {output_path.stat().st_size:,} bytes")
    print("\nThis PDF includes:")
    print("  - Spread visualization")
    print("  - Card-by-card interpretations")
    print("  - Full synthesized Reading section")
else:
    print("No synthesized reading available (API key not set)")

# Lets create a PDF for the "Kate Signature" version, too
if kate_reading:
    output_path = Path("reading_synthesized_kate.pdf")
    render_reading_to_pdf(
        reading=kate_reading,  # SynthesizedReading includes the narrative
        output_path=output_path,
        title="Your Creative Journey",
        layout_positions=layout_positions,
    )
    print()
    print(f"PDF generated: {output_path}")
    print(f"Size: {output_path.stat().st_size:,} bytes")

PDF generated: reading_synthesized.pdf
Size: 45,222 bytes

This PDF includes:
  - Spread visualization
  - Card-by-card interpretations
  - Full synthesized Reading section

PDF generated: reading_synthesized_kate.pdf
Size: 52,431 bytes


## Using Local LLMs (Ollama)

You can also use local models via Ollama or any OpenAI-compatible server:

In [ ]:
from arcanite.interpretation.llm import LocalProvider

# Example: Using Ollama (if running locally)
# local_provider = LocalProvider(
#     model="llama3.2",
#     base_url="http://localhost:11434/v1",
# )
#
# local_synthesizer = ReadingSynthesizer(
#     provider=local_provider,
#     tradition="intuitive",
# )
#
# local_reading = await local_synthesizer.synthesize(reading, context)

print("To use Ollama:")
print("1. Install Ollama: brew install ollama")
print("2. Pull a model: ollama pull llama3.2")
print("3. Uncomment the code above and run!")

To use Ollama:
1. Install Ollama: brew install ollama
2. Pull a model: ollama pull llama3.2
3. Uncomment the code above and run!


## Summary

The Arcanite pipeline:

```
TarotDeck.load()           # Load 78 cards with rich metadata
        │
        ▼
create_reading()           # Draw cards for a spread
        │
        ▼
assemble_context()         # Layer 1: Deterministic RAG assembly
        │
        ├─────────────────► render_reading_to_pdf()  # Deterministic PDF
        │
        ▼
synthesizer.synthesize()   # Layer 2: LLM narrative synthesis
        │
        ▼
render_reading_to_pdf()    # Full PDF with synthesis
```

Key features:
- **RAG mapping** means the LLM doesn't hallucinate card meanings
- **Tradition prompts** customize the reading style
- **Protocol-based design** supports future oracle systems (Lenormand, etc.)
- **PDF generation** via Typst produces beautiful reports